## Attention-Model

This network spreading model considers the role of attention span in modeling the spread of memes on social networks. Agents maintain a limited attention span (queue) of their most recently observed memes. They draw from this queue to re-share memes.

In [22]:
from queue import Queue 

In [23]:
class Agent:
    
    def __init__(self,ID,attention_span,interests):
        self.ID = ID
        self.q = Queue(maxsize = attention_span) 
        self.interests = interests
        
    def add_meme(self,meme):
        if(self.q.full()):
            self.q.get()
        self.q.put(meme)
        
    def select_meme(self):
        if(self.q.empty()):
            return None
        #return item from queue at random, weighted by quality and similarity to agent interests
                
#Consider adding some variable(s) to capture agent interests/topics. 
#This could influence which meme in queue they select to share.

#When agents share memes, that meme gets added to the queue of their neighbors.
#The shared meme also remains in the queue of the agent who shared it. (thoughts on this?)

In [24]:
class Meme:
    
    def __init__(self,quality,topic):
        self.quality = quality
        self.topic = topic
    

#Memes could have some quality/novelty attribute and some topic attribute pulled from some distributions.
#These would be considered by the agent when selected which meme to share.